# Анализ медицинских данных Diabetes с помощью GigaChat

## Авторизация

Регестрируемся на сайте developers.sber.ru, переходим в GigaChat API, подключаем тариф Freemium, переходим на вкладку "Настройка API", копируем Client ID и ключ авторизации в вкладку "Секреты" левой панели Colab для хранения ключей

In [2]:
from google.colab import userdata
client_id = userdata.get('SBER_ID')
auth = userdata.get('SBER_AUTH')

### Получим токен доступа для авторизации запросов к *API*

In [3]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    rq_uid = str(uuid.uuid4())

    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [4]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.lVOeVIbw4HwnTINKsDNLVrj1pFS2rN8yF59E2koc0PYovORNDWlYHnMYzT_JxUDozr-YH6Jo-0H4IiZRluSjd3Maq4I9PJBTt9ybtoEF1-I9Xau9SJNK6MK_eNP5GvBv-abGR76A1qVzyxIy_0P2BEVOBNTLMrqGvmWz86FTGQ7Pnl3BTgy8cpCRpuKkt1NmiOJZWMTMaC4-48bfSsX3PZFb5HCKjvoh-sC3cHYALtkP_4dIq44lFzdGWIB6w9QiXSQayklva2Xx8HH5woUSBbYlbcwU0sTkmpfYHXaxPxVwUn1F2EWtnbMmS56NcT7_PYlZy9P7bfJEEruaLD1tUQ.B703h940JAFd5m0awMwk-A.Ns5M_iEr34l-ZPRTWdSKyK0r6R6pV7pAq0BN81pep7u_5Q640Ci92gz8KqxZFeqxf-6goi_JL6ZGm-qPdY49DoF7qTYUG_Y3Cp1DjH6kyvXF2GL6r2v5kAXwbLXMyL-Zl_gX971L6QJwB46WECDqLuOt0lP7IXR71DE8m6LrkWKyKc9Wi4G8yoek739AqrhSG3bIVPvS-g0XVEcyS_JhuYZY9fewXt4nVbATrjtRFIur2yJi7MLgvulrWoXhvKpqKSf4-fxUKkPYLbjYcASCjFTpNczK4_Ao8eYsZqTH2rt7Y1bXjYbls5_JX6h-cFiu1YcGqC9zh_VaIxoDoUnVmYuYHJj0ZawEPTWXKfC7agnkdlrZcPKz7-afZOtWXH-un_5YHiFFvOMPmHHmH9LOyucKw2DfbFnUDmgpRAkN6IYJ4V_HyYQorhATQebJsrNT7lGA8U2DwEtmOFZ4oCg09m8jVNnQwK403R8-dyRqhyxMNLixXSH5YU87eWBuVv6iUo3B6-1zwdarylo1d1XU19ElbiSrA

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Функция для отправки запроса к API чата и получения ответа

In [34]:
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 1024,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1

### Загрузка и исследование данных

Загрузка датасета Diabetes из sklearn, анализ структуры, вычисление корреляций с целевой переменной

In [35]:
import pandas as pd
import requests
import json
from datetime import datetime
from sklearn.datasets import load_diabetes
import numpy as np

def create_diabetes_dataframe():
    """Создание DataFrame из датасета Diabetes"""
    diabetes = load_diabetes()

    df_diabetes = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

    df_diabetes['target'] = diabetes.target

    return df_diabetes, diabetes

df_diabetes, diabetes_raw = create_diabetes_dataframe()
print("Данные Diabetes:")
print(f"Форма данных: {df_diabetes.shape}")
print(f"Колонки: {list(df_diabetes.columns)}")
print("\nПервые 5 строк:")
print(df_diabetes.head())
print("\nОсновная статистика:")
print(df_diabetes.describe())

numeric_columns = df_diabetes.select_dtypes(include=[np.number]).columns
correlations = df_diabetes[numeric_columns].corr()['target'].sort_values(ascending=False)

print(f"\nКорреляция с целевой переменной:")
print(correlations)

Данные Diabetes:
Форма данных: (442, 11)
Колонки: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'target']

Первые 5 строк:
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.046641   135.0  

Основная статистика:
                age           sex           bmi            bp            s1  \
count  4.420000e+02  4.420000e+02  4.420000e+02  4.4200

### Использование ИИ для извлечения медицинских insights


In [36]:
def analyze_data_with_gigachat(auth_token, data_description, question):
    """Использование GigaChat для анализа данных"""
    prompt = f"""
    У меня есть медицинский датасет Diabetes: {data_description}

    Вопрос: {question}

    Пожалуйста, предоставь краткий анализ и рекомендации на основе этих данных.
    """

    response = get_chat_completion(auth_token, prompt)
    if response != -1:
        return response.json()['choices'][0]['message']['content']
    else:
        return "Ошибка при обращении к GigaChat"

print("Анализ данных Diabetes с помощью GigaChat:")

data_description = f"""
Diabetes Dataset:
- Количество пациентов: {len(df_diabetes)}
- Количество признаков: {len(diabetes_raw.feature_names)}
- Признаки: {list(diabetes_raw.feature_names)}
- Целевая переменная: количественная оценка прогрессирования диабета
- Диапазон целевой переменной: от {df_diabetes['target'].min():.2f} до {df_diabetes['target'].max():.2f}
- Среднее значение целевой переменной: {df_diabetes['target'].mean():.2f}

Корреляции с целевой переменной:
{correlations.to_string()}

Статистика по ключевым признакам:
- BMI (индекс массы тела): среднее {df_diabetes['bmi'].mean():.3f}
- BP (артериальное давление): среднее {df_diabetes['bp'].mean():.3f}
- Возраст: среднее {df_diabetes['age'].mean():.3f}
- S5 (глюкоза): среднее {df_diabetes['s5'].mean():.3f}
"""

analysis_result = analyze_data_with_gigachat(
    giga_token,
    data_description,
    "Ориентируясь на датасет Diabetes ответь на следующие вопросы. Какие медицинские insights можно извлечь из этих данных? Какие признаки наиболее сильно влияют на прогрессирование диабета? Какие рекомендации можно дать для дальнейшего исследования?"
)

print(analysis_result)

Анализ данных Diabetes с помощью GigaChat:
### Анализ данных и медицинские insights

#### 1. **Признаки, сильно коррелирующие с целевой переменной (прогрессией диабета)**

- **BMI (индекс массы тела)** — коэффициент корреляции 0.586
- **S5 (уровень глюкозы)** — коэффициент корреляции 0.566
- **Артериальное давление (BP)** — коэффициент корреляции 0.441
- **S4** — коэффициент корреляции 0.430
- **S6** — коэффициент корреляции 0.382

Эти признаки являются наиболее значимыми факторами, влияющими на прогрессию диабета. Из них наиболее сильными корреляциями обладают:
- Индекс массы тела (BMI)
- Уровень глюкозы (S5)
- Артериальное давление (BP)

#### 2. **Медицинские insights**

- **Индекс массы тела (BMI)** — высокий индекс массы тела (ожирение) является одним из основных факторов риска развития диабета 2 типа. Избыточный вес увеличивает нагрузку на поджелудочную железу, что приводит к нарушению метаболизма глюкозы.
- **Уровень глюкозы (S5)** — высокий уровень глюкозы в крови является ключе

### Расширенный статистический анализ


Группировки:

*   По возрасту: 5 категорий (Молодой → Престарелый)
*   По BMI: 3 категории (Низкий → Высокий)


In [37]:
def perform_advanced_analysis(df):
    """Проведение расширенного анализа данных"""
    print("\n" + "="*50)
    print("Расширенный анализ данных Diabets:")

    age_categories = pd.cut(df['age'], bins=5, labels=['Молодой', 'Средний', 'Старший', 'Пожилой', 'Престарелый'])
    bmi_categories = pd.cut(df['bmi'], bins=3, labels=['Низкий', 'Средний', 'Высокий'])

    age_analysis = df.groupby(age_categories).agg({
        'target': ['mean', 'std', 'count'],
        'bmi': 'mean',
        'bp': 'mean'
    }).round(3)

    age_analysis_simple = {}
    for idx in age_analysis.index:
        age_analysis_simple[str(idx)] = {
            'target_mean': float(age_analysis.loc[idx, ('target', 'mean')]),
            'target_std': float(age_analysis.loc[idx, ('target', 'std')]),
            'count': int(age_analysis.loc[idx, ('target', 'count')]),
            'bmi_mean': float(age_analysis.loc[idx, ('bmi', 'mean')]),
            'bp_mean': float(age_analysis.loc[idx, ('bp', 'mean')])
        }

    print("Анализ по возрастным группам:")
    print(age_analysis)

    bmi_analysis = df.groupby(bmi_categories).agg({
        'target': ['mean', 'std'],
        'age': 'mean',
        'bp': 'mean'
    }).round(3)

    bmi_analysis_simple = {}
    for idx in bmi_analysis.index:
        bmi_analysis_simple[str(idx)] = {
            'target_mean': float(bmi_analysis.loc[idx, ('target', 'mean')]),
            'target_std': float(bmi_analysis.loc[idx, ('target', 'std')]),
            'age_mean': float(bmi_analysis.loc[idx, ('age', 'mean')]),
            'bp_mean': float(bmi_analysis.loc[idx, ('bp', 'mean')])
        }

    print("\nАнализ по категориям BMI:")
    print(bmi_analysis)

    top_correlations = correlations.abs().sort_values(ascending=False).head(6)
    print(f"\nТоп-5 наиболее коррелирующих признаков с целевой переменной:")
    for feature, corr in top_correlations.items():
        if feature != 'target':  # Исключаем сам target
            print(f"{feature}: {corr:.3f}")

    return age_analysis_simple, bmi_analysis_simple, age_analysis, bmi_analysis

age_analysis_simple, bmi_analysis_simple, age_analysis_full, bmi_analysis_full = perform_advanced_analysis(df_diabetes)


Расширенный анализ данных Diabets:
Анализ по возрастным группам:
              target                  bmi     bp
                mean     std count   mean   mean
age                                             
Молодой      133.549  67.148    51 -0.026 -0.028
Средний      136.660  76.712   103  0.001 -0.014
Старший      152.303  80.498   145 -0.001  0.002
Пожилой      167.034  75.276   117  0.008  0.017
Престарелый  181.885  67.419    26  0.017  0.024

Анализ по категориям BMI:
          target            age     bp
            mean     std   mean   mean
bmi                                   
Низкий   116.274  58.183 -0.008 -0.015
Средний  184.039  72.866  0.010  0.015
Высокий  265.625  54.479  0.001  0.039

Топ-5 наиболее коррелирующих признаков с целевой переменной:
bmi: 0.586
s5: 0.566
bp: 0.441
s4: 0.430
s3: 0.395


/tmp/ipython-input-3272489251.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_analysis = df.groupby(age_categories).agg({
/tmp/ipython-input-3272489251.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bmi_analysis = df.groupby(bmi_categories).agg({


Ключевые находки:

*   Прогрессирование диабета увеличивается с возрастом
*   Высокий BMI ассоциирован с более высокими значениями target (265.6)


### Генерация комплексного отчета

Структура отчета:

*   Медицинское введение и характеристики датасета
*   Детальный анализ по возрастным группам
*   Выводы и рекомендации


In [38]:
def generate_comprehensive_report(auth_token, diabetes_data, advanced_analysis):
    """Генерация комплексного отчета с использованием pandas и GigaChat"""

    age_analysis_simple, bmi_analysis_simple = advanced_analysis

    pandas_analysis = {
        'total_records': len(diabetes_data),
        'columns': list(diabetes_data.columns),
        'numeric_stats': {
            'target': {
                'mean': float(diabetes_data['target'].mean()),
                'std': float(diabetes_data['target'].std()),
                'min': float(diabetes_data['target'].min()),
                'max': float(diabetes_data['target'].max())
            },
            'bmi': {
                'mean': float(diabetes_data['bmi'].mean()),
                'std': float(diabetes_data['bmi'].std())
            },
            'bp': {
                'mean': float(diabetes_data['bp'].mean()),
                'std': float(diabetes_data['bp'].std())
            }
        },
        'correlation_with_target': diabetes_data.corr()['target'].to_dict(),
        'age_group_analysis': age_analysis_simple,
        'bmi_analysis': bmi_analysis_simple
    }

    report_prompt = f"""
    Сгенерируй профессиональный медицинский отчет на основе анализа датасета Diabetes:

    1. Медицинские данные (Diabetes Dataset):
    - Количество пациентов: {pandas_analysis['total_records']}
    - Признаки: {list(diabetes_raw.feature_names)}
    - Статистика по целевой переменной: Среднее {pandas_analysis['numeric_stats']['target']['mean']:.2f}, Стандартное отклонение {pandas_analysis['numeric_stats']['target']['std']:.2f}
    - Корреляции с целевой переменной: BMI ({pandas_analysis['correlation_with_target']['bmi']:.3f}), BP ({pandas_analysis['correlation_with_target']['bp']:.3f}), S5 ({pandas_analysis['correlation_with_target']['s5']:.3f})

    2. Анализ по возрастным группам: {json.dumps(pandas_analysis['age_group_analysis'], ensure_ascii=False, indent=2)}

    3. Анализ по категориям BMI: {json.dumps(pandas_analysis['bmi_analysis'], ensure_ascii=False, indent=2)}

    Сделай медицинские выводы о факторах риска развития диабета и предложи рекомендации для дальнейшего исследования.
    """

    giga_report = get_chat_completion(auth_token, report_prompt)
    if giga_report != -1:
        return giga_report.json()['choices'][0]['message']['content']
    else:
        return "Ошибка при генерации отчета"

def save_diabetes_results(dataframe, analysis_text, advanced_analysis, filename_prefix):
    """Сохранение результатов анализа Diabetes в различные форматы"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Распаковываем advanced_analysis для использования в TXT файле
    age_analysis_simple, bmi_analysis_simple = advanced_analysis

    csv_filename = f"{filename_prefix}_{timestamp}.csv"
    dataframe.to_csv(csv_filename, index=False, encoding='utf-8')

    txt_filename = f"{filename_prefix}_analysis_{timestamp}.txt"
    with open(txt_filename, 'w', encoding='utf-8') as f:
        f.write("Анализ данных Diabetes\n")
        f.write("=" * 50 + "\n\n")
        f.write(analysis_text)
        f.write("\n\nРасширенная статистика:\n")
        f.write("Анализ по возрастным группам:\n")
        f.write(str(age_analysis_full))
        f.write("\n\nАнализ по категориям BMI:\n")
        f.write(str(bmi_analysis_full))
        f.write(f"\n\nКорреляции с целевой переменной:\n{correlations.to_string()}")

    json_data = {
        'metadata': {
            'generated_at': timestamp,
            'dataset': 'Diabetes',
            'records_count': len(dataframe),
            'features': list(diabetes_raw.feature_names),
            'description': diabetes_raw.DESCRIPTION[:200] + "..." if hasattr(diabetes_raw, 'DESCRIPTION') else "Diabetes dataset from sklearn"
        },
        'correlation_analysis': dataframe.corr()['target'].to_dict(),
        'summary_stats': {
            'target_mean': float(dataframe['target'].mean()),
            'target_std': float(dataframe['target'].std()),
            'bmi_mean': float(dataframe['bmi'].mean()),
            'bp_mean': float(dataframe['bp'].mean()),
            'age_mean': float(dataframe['age'].mean())
        },
        'analysis': analysis_text,
        'advanced_analysis': {
            'age_groups': age_analysis_simple,
            'bmi_categories': bmi_analysis_simple
        }
    }

    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

    print(f"\nРезультаты сохранены в файлы:")
    print(f"- CSV: {csv_filename}")
    print(f"- TXT: {txt_filename}")
    print(f"- JSON: {json_filename}")

print("Комплексный медицинский отчет:")
final_report = generate_comprehensive_report(giga_token, df_diabetes, (age_analysis_simple, bmi_analysis_simple))
print(final_report)

save_diabetes_results(df_diabetes, final_report, (age_analysis_simple, bmi_analysis_simple), "diabetes_analysis_report")


Комплексный медицинский отчет:
# Медицинский отчет по анализу датасета Diabetes

## Введение

В представленном датасете Diabetes содержится информация о 442 пациентах, страдающих сахарным диабетом. Исследование включает анализ медицинских признаков, таких как возраст, пол, индекс массы тела (BMI), артериальное давление (BP), а также уровни глюкозы в крови (s1, s2, s3, s4, s5, s6). Целью данного отчета является выявление факторов риска развития сахарного диабета и разработка рекомендаций для дальнейшего исследования.

## Основные характеристики датасета

- **Общее количество пациентов:** 442
- **Признаки:** ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
- **Статистика по целевой переменной (уровень глюкозы в крови):**
  - Среднее значение: 152.13
  - Стандартное отклонение: 77.09
- **Корреляции с целевой переменной:**
  - Индекс массы тела (BMI): 0.586
  - Артериальное давление (BP): 0.441
  - s5: 0.566

## Анализ по возрастным группам

На основе анализа датасета были в